# Importing Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications import InceptionResNetV2, DenseNet201, ResNet152

# Image Preprocessing

In [ ]:
def train_val_generators(TRAINING_DIR, TESTING_DIR, VALIDATION_DIR):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=180,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range = 0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      class_mode='categorical',
                                                      target_size=(100, 100))
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(directory=TESTING_DIR,
                                                                class_mode='categorical',
                                                                target_size=(100, 100))
    validation_datagen = ImageDataGenerator(rescale = 1./255)
    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                 class_mode = 'categorical',
                                                                 target_size=(100, 100))
    return train_generator, test_generator, validation_generator

In [ ]:
training = "/kaggle/input/plants-classification/train"
testing = "/kaggle/input/plants-classification/test"
validation = "/kaggle/input/plants-classification/val"
train_generator, test_generator, validation_generator = train_val_generators(training, testing, validation)

# Image Visualization

In [ ]:
x_train = []
c = 0
for feature, label in train_generator:
    x_train.append(np.array(feature))
    c += 1
    if c == 1:
        break

x_train = np.array(x_train)
print(x_train.shape)
x_train = np.reshape(x_train, (32, 100, 100, 3))
print(x_train.shape)

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(30, 30))
for i in range(0, 16):
    ax[int(i / 4), (i % 4)].imshow(x_train[i])

# Model Creation

In [ ]:
def create_model():
    model = models.Sequential([
      layers.Conv2D(32, (2,2), activation='relu', input_shape=(100, 100, 3)),
      layers.MaxPooling2D(2,2),
      layers.Conv2D(64, (2,2), activation='relu'),
      layers.MaxPooling2D(2,2),
      layers.Conv2D(64, (3,3), activation='relu'),
      layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
      layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      layers.Flatten(),
      # 512 neuron hidden layer
        layers.Dropout(0.1),
      layers.Dense(512, activation='relu'),
        layers.Dropout(0.1),
      layers.Dense(30, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

# Model Training

In [ ]:
model = create_model()
epochs = 10
history = model.fit(train_generator,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator)

# Plotting training and testing accuracies

In [ ]:
def plot_accuracies(history):
    acc=history.history['accuracy']
    val_acc=history.history['val_accuracy']
    loss=history.history['loss']
    val_loss=history.history['val_loss']

    epochs=range(len(acc)) # Get number of epochs

    #------------------------------------------------
    # Plot training and validation accuracy per epoch
    #------------------------------------------------
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and testing accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(['Training', 'testing'])
    plt.show()
    print("")

    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and testing loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(['Training', 'testing'])
    plt.show()

In [ ]:
plot_accuracies(history)

# Using keras inbuilt models

In [ ]:
resnet_model = models.Sequential()

pretrained_models = [
    InceptionResNetV2(include_top = False,input_shape = (100, 100, 3),pooling='max', classes = 30,weights = 'imagenet', classifier_activation="softmax",),
    DenseNet201(include_top=False,weights="imagenet",input_shape=(100, 100, 3),pooling='max',classes=30,classifier_activation="softmax"),
    ResNet152(include_top=False, weights="imagenet",input_shape=(100, 100, 3),pooling='max',classes=30, classifier_activation="softmax"),
]

final_models = []

for model in pretrained_models:
    for layer in model.layers:
        layer.trainable = False
    m = models.Sequential()   
    m.add(model)
    m.add(layers.Flatten())
    m.add(layers.Dense(512, activation = 'relu'))
    m.add(layers.Dense(30, activation = "softmax"))
    final_models.append(m)

In [ ]:
for model in final_models:
    print(model.summary())

In [ ]:
for model in final_models:
    model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
for model in final_models:
    history = model.fit(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator)
    plot_accuracies(history)